# Project Layout Tool with E2B Sandbox

Demonstration of the project_layout tool functionality using E2B sandbox to explore directory structure of a cloned repository.

In [1]:
// Import E2B SDK and load environment
import { Sandbox } from "https://esm.sh/e2b@latest";
import * as dotenv from "jsr:@std/dotenv";

import { projectLayoutTool, handleProjectLayout } from '../src/tools/project_layout.ts';
import { StreamData } from 'npm:ai';

// Load environment variables
await dotenv.load({ export: true });

const apiKey = Deno.env.get("E2B_API_KEY");
if (!apiKey) {
  throw new Error("E2B_API_KEY environment variable is required");
}

console.log("Environment loaded successfully");

// Set up mock objects for testing
const mockData = new StreamData();
const mockMessages: any[] = [];
const mockAppendToolCallMessage = (msg: any) => [msg];

console.log('Tool definition:', projectLayoutTool);

Environment loaded successfully
Tool definition: {
  type: "function",
  function: {
    name: "project_layout",
    description: "Get the full directory structure of the repository as text.",
    parameters: {
      type: "object",
      properties: {
        rootPath: {
          type: "string",
          description: "Root directory path to start traversal from (default: current working directory)"
        },
        maxDepth: {
          type: "number",
          description: "Maximum depth to traverse (default: 10)",
          default: 10
        },
        includeHidden: {
          type: "boolean",
          description: "Include hidden files and directories (default: false)",
          default: false
        }
      },
      required: []
    }
  }
}


In [2]:
// Create sandbox and clone repository
const sandbox = await Sandbox.create({
    template: "base",
    apiKey: apiKey,
    timeoutMs: 600_000 // 10 minutes
  });
  
  console.log(`Sandbox created with ID: ${sandbox.id}`);
  
  // Clone the Sudoku Solver repository
  const cloneResult = await sandbox.commands.run("git clone https://github.com/Rahimeen-Altaf/Sudoku-Solver-DSA.git /tmp/sudoku-solver");
  console.log("Repository cloned successfully");
  
  // Configure git
  await sandbox.commands.run("git config --global user.email 'test@example.com'");
  await sandbox.commands.run("git config --global user.name 'Test User'");
  
  console.log("Sandbox ready for testing");

Sandbox created with ID: undefined
Repository cloned successfully
Sandbox ready for testing


In [3]:
// Test layout of the entire /tmp directory to show the sandbox structure
const tmpToolCall = {
  id: 'test-tmp-layout',
  func: {
    name: 'project_layout',
    arguments: {
      rootPath: '/tmp',
      maxDepth: 3,
      includeHidden: false
    }
  }
};

console.log('=== /tmp Directory Structure ===');
const tmpResult = await handleProjectLayout(
  tmpToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log(tmpResult[0].tool_call_result.structure);

=== /tmp Directory Structure ===
Directory Structure:
/tmp/
└── sudoku-solver/
    ├── __pycache__/
    │   ├── puzzles.cpython-310.pyc
    │   └── solutions.cpython-310.pyc
    ├── puzzles.py
    ├── puzzles.txt
    ├── README.md
    ├── solutions.py
    └── solver.py



In [4]:
// Test with different parameters - include hidden files and deeper depth
const detailedToolCall = {
  id: 'test-detailed-layout',
  func: {
    name: 'project_layout',
    arguments: {
      rootPath: '/tmp/sudoku-solver',
      maxDepth: 8,
      includeHidden: true
    }
  }
};

console.log('=== Detailed Structure (with hidden files) ===');
const detailedResult = await handleProjectLayout(
  detailedToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log(detailedResult[0].tool_call_result.structure);

=== Detailed Structure (with hidden files) ===
Directory Structure:
/tmp/sudoku-solver/
├── __pycache__/
│   ├── puzzles.cpython-310.pyc
│   └── solutions.cpython-310.pyc
├── .git/
├── puzzles.py
├── puzzles.txt
├── README.md
├── solutions.py
└── solver.py



In [5]:
// Test project layout of the cloned repository
const repositoryToolCall = {
  id: 'test-repo-layout',
  func: {
    name: 'project_layout',
    arguments: {
      rootPath: '/tmp/sudoku-solver',
      maxDepth: 5,
      includeHidden: false
    }
  }
};

console.log('=== Repository Structure ===');
const repoResult = await handleProjectLayout(
  repositoryToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log(repoResult[0].tool_call_result.structure);

=== Repository Structure ===
Directory Structure:
/tmp/sudoku-solver/
├── __pycache__/
│   ├── puzzles.cpython-310.pyc
│   └── solutions.cpython-310.pyc
├── puzzles.py
├── puzzles.txt
├── README.md
├── solutions.py
└── solver.py

